In [48]:
import numpy as np 
import pandas as pd

path='phi_070.csv'
# path='iris.csv'
df = pd.read_csv(path)

In [49]:
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,accuracy_score,ConfusionMatrixDisplay

In [50]:
x = df[['mean(nid)' ,'Median(nid)','IQR(nid)','Range(nid)']]

In [51]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df.polymer.values)
y= df['label'].values

In [52]:
scaler = StandardScaler()

In [53]:
#Doing The Train Test Split And Scaling The Data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)

In [54]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [55]:
# Converting From Numpy Array To Torch Tensor
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [56]:
#Creating Our Neural Network Model For Classification
class NeuralNetworkClassificationModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetworkClassificationModel,self).__init__()
        self.input_layer    = nn.Linear(input_dim,128)
        self.hidden_layer1  = nn.Linear(128,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()
    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

In [57]:
input_dim  = 4 
output_dim = 12
model = NeuralNetworkClassificationModel(input_dim,output_dim)

In [58]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [59]:
def train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model(X_train)

        #calculate the loss
        loss_train = criterion(output_train, y_train)
        


        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()

        
        output_test = model(X_test)
        loss_test = criterion(output_test,y_test)

        train_losses[epoch] = loss_train.item()
        test_losses[epoch] = loss_test.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Test Loss: {loss_test.item():.4f}")

In [60]:
num_epochs = 1000
train_losses = np.zeros(num_epochs)
test_losses  = np.zeros(num_epochs)

In [61]:
train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses)

Epoch 50/1000, Train Loss: 0.3809, Test Loss: 0.4126
Epoch 100/1000, Train Loss: 0.3339, Test Loss: 0.3627
Epoch 150/1000, Train Loss: 0.3159, Test Loss: 0.3690
Epoch 200/1000, Train Loss: 0.2989, Test Loss: 0.3395
Epoch 250/1000, Train Loss: 0.2848, Test Loss: 0.3105
Epoch 300/1000, Train Loss: 0.2665, Test Loss: 0.3048
Epoch 350/1000, Train Loss: 0.2457, Test Loss: 0.2950
Epoch 400/1000, Train Loss: 0.2335, Test Loss: 0.3142
Epoch 450/1000, Train Loss: 0.2172, Test Loss: 0.3138
Epoch 500/1000, Train Loss: 0.2099, Test Loss: 0.2988
Epoch 550/1000, Train Loss: 0.2098, Test Loss: 0.3007
Epoch 600/1000, Train Loss: 0.2049, Test Loss: 0.3217
Epoch 650/1000, Train Loss: 0.1975, Test Loss: 0.3569
Epoch 700/1000, Train Loss: 0.2122, Test Loss: 0.3413
Epoch 750/1000, Train Loss: 0.1825, Test Loss: 0.3795
Epoch 800/1000, Train Loss: 0.1861, Test Loss: 0.4187
Epoch 850/1000, Train Loss: 0.1724, Test Loss: 0.4317
Epoch 900/1000, Train Loss: 0.1704, Test Loss: 0.5118
Epoch 950/1000, Train Loss: 0

In [62]:
predictions_train = []
predictions_test =  []
with torch.no_grad():
    predictions_train = model(X_train)
    predictions_test = model(X_test)

In [63]:
def get_accuracy_multiclass(pred_arr,original_arr):
    if len(pred_arr)!=len(original_arr):
        return False
    pred_arr = pred_arr.numpy()
    original_arr = original_arr.numpy()
    final_pred= []
    # we will get something like this in the pred_arr [32.1680,12.9350,-58.4877]
    # so will be taking the index of that argument which has the highest value here 32.1680 which corresponds to 0th index
    for i in range(len(pred_arr)):
        final_pred.append(np.argmax(pred_arr[i]))
    final_pred = np.array(final_pred)
    count = 0
    #here we are doing a simple comparison between the predicted_arr and the original_arr to get the final accuracy
    for i in range(len(original_arr)):
        if final_pred[i] == original_arr[i]:
            count+=1
    return count/len(final_pred)

In [64]:
train_acc = get_accuracy_multiclass(predictions_train,y_train)
test_acc  = get_accuracy_multiclass(predictions_test,y_test)

In [65]:
print(f"Training Accuracy: {round(train_acc*100,3)}")
print(f"Test Accuracy: {round(test_acc*100,3)}")

Training Accuracy: 91.667
Test Accuracy: 84.167


In [66]:
# from torchmetrics import ConfusionMatrix
# from torchmetrics.classification import MulticlassConfusionMatrix

In [67]:
confusion_matrix(predictions_test, y_test)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multiclass targets

In [29]:
# target = torch.tensor([1, 1, 0, 0])
# preds = torch.tensor([0, 1, 0, 0])
# confmat = ConfusionMatrix(num_classes=2)
# confmat(preds, target)

tensor([[2, 0],
        [1, 1]])

In [ ]:
# MulticlassConfusionMatrix(num_classes, ignore_index=None, normalize=None, validate_args=True, **kwargs)